In [1]:
# import ploting tools
import matplotlib as mpl
import matplotlib.pyplot as plt

import numpy as np
import h5py
import random

# import the MUQ libraries
import muq.Utilities as mu # import MUQ utilities module
import muq.Modeling as mm # import MUQ modeling module
import muq.Approximation as ma # import MUQ approximation module
import muq.SamplingAlgorithms as ms


### 1. Define Densities

In [2]:
tgtCov = np.array([[1.0,0.3],[0.3,1.5]]) # target distribution covariance
tgtMu = np.array([.8, 2.4]) # target distribution mean
        
# Make space for a 2x2 collection of densities.  Index (1,1) should hold the true target distribution
densities = [ [None]*2 ]*2 

# Add the densities
densities[0][0] = mm.Gaussian(tgtMu, tgtCov * 2.0).AsDensity()
densities[0][1] = mm.Gaussian(tgtMu, tgtCov * 1.3).AsDensity()
densities[1][0] = mm.Gaussian(tgtMu, tgtCov * 1.5).AsDensity()
densities[1][1] = mm.Gaussian(tgtMu, tgtCov).AsDensity()

Now unroll the densities, using a multiindex set to keep track of the linear ordering

In [3]:
indices = mu.MultiIndexFactory.CreateFullTensor(orders=[len(densities)-1,len(densities[0])-1])

models = []
for i in range(indices.Size()):
    models.append( densities[ indices.at(i).GetValue(0) ][ indices.at(i).GetValue(1) ] )

### 2. Run MultiIndex MCMC

In [4]:
numFineSamps = 1000
numChains = 4

#MIMCMC
mioptions = dict()
mioptions['NumSamples_0_0'] = numFineSamps * 100 # Number of samples per level
mioptions['NumSamples_1_0'] = numFineSamps * 10
mioptions['NumSamples_0_1'] = numFineSamps * 10
mioptions['NumSamples_1_1'] = numFineSamps

mioptions['MLMCMC.Subsampling_0_0'] = 25
mioptions['MLMCMC.Subsampling_1_0'] = 5
mioptions['MLMCMC.Subsampling_0_1'] = 5
mioptions['MLMCMC.Subsampling_1_1'] = 1

mioptions['Proposal.Method'] = 'AMProposal'
mioptions['Proposal.InitialVariance'] = 2.0
mioptions['Proposal.AdaptSteps'] = 50
mioptions['Proposal.AdaptStart'] = 100

# Now, plug models into MIMCMC
chains = [None]*numChains
for i in range(numChains):
    startPt = tgtMu + 1.5*np.random.randn(2)
    mimcmc = ms.MIMCMC(mioptions, startPt, models, indices)
    chains[i] = mimcmc.Run()

### 3. Analyze the Results

In [5]:
print('Convergence diagnostic Rhat:')
print(ms.Diagnostics.Rhat(chains,{'Multivariate':True}))

Convergence diagnostic Rhat:
[1.00182023]


In [6]:
samps = chains[0]
sampMean = samps.Mean()
print(f"Sample Mean = {sampMean.transpose()}")

Sample Mean = [0.74673428 2.47915221]


In [7]:
sampVar = samps.Variance()
print(f"Sample Variance = {sampVar.transpose()}")

Sample Variance = [1.05936028 1.75953172]


In [8]:
sampCov = samps.Covariance()
print(f"Sample Covariance = {sampCov}")

Sample Covariance = [[1.05936028 0.18057371]
 [0.18057371 1.75953172]]


In [9]:
sampMom3 = samps.CentralMoment(3)
print(f"Sample Third Moment = {sampMom3}")

Sample Third Moment = [-0.12826552 -0.14462487]


#### Statistical Accuracy
  

In [10]:
batchESS = samps.ESS(method="Batch")
batchMCSE = samps.StandardError(method="Batch")

spectralESS = samps.ESS(method="Wolff")
spectralMCSE = samps.StandardError(method="Wolff")

print('ESS:')
print('  Batch:    ', batchESS)
print('  Spectral: ', spectralESS)

print('MCSE:')
print('  Batch:    ', batchMCSE)
print('  Spectral: ', spectralMCSE)

ESS:
  Batch:     [ 60.43247973 109.25383677]
  Spectral:  [ 49.77324775 115.03578275]
MCSE:
  Batch:     [0.13239959 0.12690543]
  Spectral:  [0.14588944 0.12367504]
